In [123]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time ## REMOVE THIS LATER

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [124]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-23'
INVESTMENT = 1000000


In [126]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        #only append the stock to the final stock list if it is valid
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                #if there are no valid close prices, the stock is delisted
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

def check_volume(ticker):
    volume = yf.Ticker(ticker).history(period='1d')['Volume']
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid

all_data = pd.read_csv('Tickers_Example.csv', header=None)

all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [127]:
# Create a list of all the stock betas (retrieved from yfinance) of stock tickers in a list
def get_betas(tickers):
    betas = []
    
    for ticker in tickers:
        betas.append(yf.Ticker(ticker).info.get('beta'))

    return betas

In [128]:
def get_sector(tickers):
    sectors = []

    for ticker in tickers:
        sectors.append(yf.Ticker(ticker).info.get('sector'))

    return sectors

In [129]:
# Add a column containing all the stock betas to [all_data]
all_data = all_data.assign(Beta=get_betas(all_data['Ticker']))
all_data = all_data.assign(Sector=get_sector(all_data['Ticker']))

all_data

,Ticker,Beta,Sector
0,AAPL,1.240,Technology
1,ABBV,0.613,Healthcare
2,ABT,0.722,Healthcare
3,ACN,1.245,Technology
4,AIG,1.069,Financial Services
5,AMZN,1.146,Consumer Cyclical
6,AXP,1.214,Financial Services
7,BA,1.572,Industrials
8,BAC,1.325,Financial Services
9,BB.TO,1.068,Technology


In [130]:
# Choose the max(24, data.length) stocks that match the market the closest
def filter_betas(data):
    # Remove any negative betas, as long as data ends with at least 12 characters
    data.sort_values(by='Beta', axis=0, inplace=True, kind='quicksort')
    data.drop(index=data[(data['Beta'] <= 0) & (len(data) > 12)].index, inplace=True)
    data.reset_index(drop=True, inplace=True)

    data['temp'] = abs(1-data['Beta'])
    data.sort_values(by='temp', axis=0, inplace=True, kind='quicksort')    # Sort data by temp
    data.drop(columns='temp', axis=1, inplace=True)    # Remove temp

    data.index = range(0, len(data))    # Reassign the index
    
    return data[:24] if len(data) > 24 else data    # Return first 24 elements of data if it is greater than 24, other wise return data

In [131]:
stocks = [x for _, x in all_data.groupby('Sector')]
stocks = list(map(filter_betas, stocks))
stocks

[  Ticker   Beta                  Sector
 0   T.TO  0.722  Communication Services,
   Ticker   Beta             Sector
 0   AMZN  1.146  Consumer Cyclical,
   Ticker   Beta              Sector
 0     MO  0.670  Consumer Defensive
 1     KO  0.620  Consumer Defensive
 2     PM  0.560  Consumer Defensive
 3    PEP  0.542  Consumer Defensive
 4     CL  0.415  Consumer Defensive
 5     PG  0.414  Consumer Defensive,
   Ticker   Beta              Sector
 0    USB  1.040  Financial Services
 1     BK  1.060  Financial Services
 2    AIG  1.069  Financial Services
 3  RY.TO  0.842  Financial Services
 4  TD.TO  0.822  Financial Services
 5    AXP  1.214  Financial Services
 6    BLK  1.311  Financial Services
 7    BAC  1.325  Financial Services
 8      C  1.426  Financial Services
 9   PYPL  1.436  Financial Services,
   Ticker   Beta      Sector
 0    ABT  0.722  Healthcare
 1    PFE  0.615  Healthcare
 2   ABBV  0.613  Healthcare
 3    UNH  0.591  Healthcare
 4    BMY  0.441  Healthcare
 5

In [132]:
all_data = filter_betas(all_data)
all_data.index = all_data.index + 1

In [ ]:
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')

sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')
sp500 = sp500.reindex(tsx60.index, method='nearest')

market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
market_returns = market_returns.dropna()
market_returns_mean = market_returns.mean()

market_returns_df = pd.DataFrame(market_returns)

display(market_returns_df.index)

DatetimeIndex(['2023-11-28', '2023-11-29', '2023-11-30', '2023-12-01',
               '2023-12-04', '2023-12-05', '2023-12-06', '2023-12-07',
               '2023-12-08', '2023-12-11',
               ...
               '2024-11-07', '2024-11-08', '2024-11-11', '2024-11-12',
               '2024-11-13', '2024-11-14', '2024-11-15', '2024-11-18',
               '2024-11-19', '2024-11-20'],
              dtype='datetime64[ns]', name='Date', length=247, freq=None)

In [191]:
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])

for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        ticker_returns[ticker] = data.pct_change().dropna()  # Calculate daily returns
        
ticker_returns.index = pd.to_datetime(ticker_returns.index.strftime("%Y-%m-%d"))

In [ ]:
months = market_returns_df.resample('MS').bfill().index

for month in months:
    temp = market_returns_df[(market_returns_df.index.month == month.month) & (market_returns_df.index.year == month.year)].index
    if len(temp) < 18:
        market_returns_df.drop(index=temp, inplace=True)
        ticker_returns.drop(index=temp, inplace=True)

print([val for val in ticker_returns.index if val not in market_returns_df.index])

print(len(market_returns_df))
print(len(ticker_returns))


[Timestamp('2023-12-26 00:00:00'), Timestamp('2024-05-20 00:00:00'), Timestamp('2024-07-01 00:00:00'), Timestamp('2024-08-05 00:00:00'), Timestamp('2024-10-14 00:00:00')]
230
231


In [223]:
portfolio_data = []
def portfolio_generator(tickers):
    global portfolio_data
    
    ticker_prices = {}  # Dictionary to store the closing prices
    target_date = '2024-11-05'  # Target date for historical data
    
    # Fetch the historical data for each ticker
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        history = stock.history(start=target_date, end='2024-11-06')
        time.sleep(0.3)  # Add a small delay to avoid rate limiting
        
        # Fetch the closing price for the target date
        if not history.empty and target_date in history.index.strftime('%Y-%m-%d'):
            close_price = history.loc[history.index.strftime('%Y-%m-%d') == target_date, 'Close'].values[0]
            ticker_prices[ticker] = close_price
        else:
            ticker_prices[ticker] = None  # No data available for that date
    
    # Optimization setup for portfolio sizes between 12 and 24
    for i in range(12, 25):
        selected_tickers = tickers[:i]  # Select the first i tickers
        
        # Define the objective function for optimization
        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # Initial weights (equal distribution for the selected tickers)
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # Constraint to ensure the sum of weights is 1
        def sum_weights(weights):
            return sum(weights) - 1
        
        # Constraints for the optimization
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # Bounds for each stock: minimum weight of 1/2n% and max 15% weight
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # Perform optimization
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x  # Get the optimized weights
            
            # Initialize variables for storing results
            total_costs = []
            shares_purchased = []
            inv = 1000000  # Investment amount
            
            # Calculate portfolio value and shares purchased, along with fees
            portfolio_val = 1000000 * weights  # Portfolio value based on optimized weights
            shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares
            
            # Calculate fees: minimum of $3.95 or $0.001 per share * number of shares
            fee = np.minimum(3.95, 0.001 * shares)
            
            # Subtract the fees from the portfolio value
            portfolio_val -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers),  # Tickers in the portfolio
                'weights': result.x,  # Optimized weights
                'STD Between Stock and Market Index': result.fun,  # Standard deviation (risk)
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_val.sum(),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_val.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    # Return the final portfolio data as a DataFrame
    return pd.DataFrame(portfolio_data)

# Example usage
  # Initialize the global portfolio data list
portfolio_df = portfolio_generator(all_data['Ticker'])  # Assuming all_data['Ticker'] is a list of tickers
portfolio_df


,tickers,weights,STD Between Stock and Market Index,num_stocks,total_cost,shares_purchased,portfolio_value,final_fees
0,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.041666666666666685, 0.04166666666666667, 0....",0.010090,12,999987.616587,"[315.1485102731414, 205.801962288549, 875.1662...",999987.616587,12.383413
1,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.038461538461538464, 0.038461538461538464, 0...",0.009796,13,999988.050783,"[290.90631717520733, 189.97104211250675, 807.8...",999988.050783,11.949217
2,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03571428571428572, 0.03571428571428571, 0.0...",0.009649,14,999988.526458,"[270.1272945198354, 176.40168196161338, 750.14...",999988.526458,11.473542
3,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03333333333333333, 0.03333333333333333, 0.0...",0.009257,15,999985.651216,"[252.11880821851304, 164.6415698308392, 700.13...",999985.651216,14.348784
4,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.03125000000000001, 0.03125000000000001, 0.0...",0.008987,16,999985.456559,"[236.361382704856, 154.35147171641177, 656.374...",999985.456559,14.543441
5,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.029411764705882353, 0.029411764705882356, 0...",0.009034,17,999985.493941,"[222.45777195751148, 145.27197338015225, 617.7...",999985.493941,14.506059
6,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.027777777777777776, 0.027777777777777787, 0...",0.009032,18,999985.676113,"[210.09900684876084, 137.20130819236604, 583.4...",999985.676113,14.323887
7,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.02631578947368421, 0.026315789473684213, 0....",0.009038,19,999985.192129,"[199.0411643830366, 129.98018670855726, 552.73...",999985.192129,14.807871
8,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.025, 0.025000000000000015, 0.02500000000000...",0.008843,20,999983.827118,"[189.08910616388476, 123.48117737312946, 525.0...",999983.827118,16.172882
9,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.023809523809523808, 0.023809523809523808, 0...",0.008642,21,999982.878115,"[180.08486301322358, 117.60112130774228, 500.0...",999982.878115,17.121885


In [220]:
def portfolio_generator(tickers):
    global portfolio_data
    
    ticker_prices = {}  # Dictionary to store the closing prices
    target_date = '2024-11-05'  # Target date for historical data
    
    # Fetch the historical data for each ticker
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        history = stock.history(start=target_date, end='2024-11-06')
        time.sleep(0.3)  # Add a small delay to avoid rate limiting
        
        # Fetch the closing price for the target date
        if not history.empty and target_date in history.index.strftime('%Y-%m-%d'):
            close_price = history.loc[history.index.strftime('%Y-%m-%d') == target_date, 'Close'].values[0]
            ticker_prices[ticker] = close_price
        else:
            ticker_prices[ticker] = None  # No data available for that date
    
    # Optimization setup for portfolio sizes between 12 and 24
    for i in range(12, 25):
        selected_tickers = tickers[:i]  # Select the first i tickers
        
        # Define the objective function for optimization
        def objective(weights):
            # Calculate the investment allocation per stock
            allocations = weights * 1000000  # Total budget: $1,000,000
            
            # Calculate fees for each allocation
            fees = [
                min(3.95, 0.001 * (allocation / ticker_prices[ticker]))
                if ticker_prices[ticker] is not None else 0
                for allocation, ticker in zip(allocations, selected_tickers)
            ]
            
            # Effective cost after fees
            effective_costs = [
                (allocation / ticker_prices[ticker]) + fee
                if ticker_prices[ticker] is not None else 0
                for allocation, fee, ticker in zip(allocations, fees, selected_tickers)
            ]
            
            # Add risk to the fees-adjusted portfolio value (proxy for minimizing fees-adjusted variance)
            return np.std(np.array(effective_costs))
        
        # Initial weights (equal distribution for the selected tickers)
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # Constraint to ensure the sum of weights is 1
        def sum_weights(weights):
            return sum(weights) - 1
        
        # Constraints for the optimization
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # Bounds for each stock: minimum weight of 1/2n% and max 15% weight
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # Perform optimization
        result1 = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        initial_weights_stage2 = result1.x
        
        # Stage 2: Adjust for Fees and Remaining Budget
        def result2_objective(weights):
            total_cost = 0
            investment = 1_000_000  # Total budget in CAD
            
            for weight, ticker in zip(weights, selected_tickers):
                if ticker_prices.get(ticker) is not None:
                    # Calculate number of shares
                    allocated_investment = investment * weight
                    num_shares = allocated_investment / ticker_prices[ticker]
                    
                    # Calculate fees
                    fee = min(3.95, 0.001 * num_shares)
                    total_cost += num_shares * ticker_prices[ticker] + fee
            
            # Return negative remaining budget (we want to fully utilize budget)
            return abs(1_000_000 - total_cost)
        
        result2 = minimize(
            result2_objective,
            initial_weights_stage2,
            constraints=constraints,
            bounds=bounds,
            options={"disp": True}
        )

        if result2.success:
            weights = result2.x  # Get the optimized weights
            
            # Calculate the actual portfolio composition after optimization
            shares_purchased = []
            total_costs = []
            inv = 1000000  # Total investment budget
            
            for weight, ticker in zip(weights, selected_tickers):
                if ticker_prices.get(ticker) is not None:  # Ensure the ticker has a valid price
                    allocated_investment = inv * weight
                    num_shares = allocated_investment / ticker_prices[ticker]
                    
                    # Calculate the fee for this stock
                    fee = min(3.95, 0.001 * num_shares)
                    
                    # Adjust the total cost including the fee
                    total_cost = (num_shares * ticker_prices[ticker]) + fee
                    
                    # Store results
                    shares_purchased.append(num_shares)
                    total_costs.append(total_cost)
                    
                    # Update the remaining budget
                    inv -= total_cost
                else:
                    shares_purchased.append(0)
                    total_costs.append(0)
            
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers),  # Tickers in the portfolio
                'weights': result2.x,  # Optimized weights
                'STD Between Stock and Market Index': result2.fun,  # Standard deviation (risk)
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_costs': total_costs,  # Costs after fees
                'shares_purchased': shares_purchased  # Final number of shares purchased
            })

    # Return the final portfolio data as a DataFrame
    return pd.DataFrame(portfolio_data)

df = portfolio_generator(all_data['Ticker'])
df

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.345847274060361
            Iterations: 6
            Function evaluations: 26
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.162092987447977
            Iterations: 2
            Function evaluations: 29
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.807125723687932
            Iterations: 2
            Function evaluations: 30
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.088583437376656
            Iterations: 2
            Function evaluations: 33
            Gradient evaluations: 2
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.029199238517322
            Iterations: 2
            Function evaluations: 35

c:\Users\sharu\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\sharu\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
c:\Users\sharu\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:495: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])


,tickers,weights,STD Between Stock and Market Index,num_stocks,total_costs,shares_purchased
0,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.08333333333333333, 0.08333333333333333, 0.0...",7105.579317,12,NaN,NaN
1,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.07142857142857142, 0.07142857142857142, 0.0...",5690.355278,14,NaN,NaN
2,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.062...",4676.586373,16,NaN,NaN
3,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.058823529411764705, 0.058823529411764705, 0...",4282.537141,17,NaN,NaN
4,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.05263157894736842, 0.05263157894736842, 0.0...",3638.728652,19,NaN,NaN
5,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...",3370.450436,20,NaN,NaN
6,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.045454545454545456, 0.045454545454545456, 0...",2925.073398,22,NaN,NaN
7,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.08333333333333333, 0.08333333333333333, 0.0...",7105.579317,12,"[83333.96363035387, 76389.21366747527, 70024.5...","[630.2970205462825, 377.3033380962791, 1470.76..."
8,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.07142857142857142, 0.07142857142857142, 0.0...",5690.355278,14,"[71429.11168316046, 66326.81962556444, 61590.1...","[540.2545890396707, 327.6029330393599, 1293.61..."
9,"UPS, TXN, USB, UNP, BK, BB.TO, AIG, CAT, AMZN,...","[0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.062...",4676.586373,16,"[62500.47272276541, 58594.0098636907, 54932.74...","[472.72276540971194, 289.4088635373629, 1153.7..."


In [ ]:
            # weights = result.x  # Get the optimized weights
            
            # Initialize variables for storing results
            # total_costs = []
            # shares_purchased = []
            
            # # Loop through each ticker and calculate costs
            # for weight, ticker in zip(weights, selected_tickers):
            #     if ticker_prices.get(ticker) is not None:  # Ensure the ticker has a valid price
            #         # Investment allocated for this stock
            #         allocated_investment = inv * weight
                    
            #         # Calculate the number of shares you can buy (allow fractional shares)
            #         num_shares = allocated_investment / ticker_prices[ticker]
                    
            #         # Calculate the fee: the smaller of $3.95 or $0.001 per share * number of shares
            #         fee_per_share = 0.001 * num_shares
            #         fee = min(3.95, fee_per_share)  # Apply the smaller fee
                    
            #         # Calculate the total cost for this stock (price * shares + fee)
            #         total_cost = (num_shares * ticker_prices[ticker]) + fee
                    
            #         # Add to results
            #         total_costs.append(total_cost)
            #         shares_purchased.append(num_shares)
                    
            #         # Update the remaining budget after deducting the total cost for this stock
            #         inv -= total_cost  # Decrease the investment by the total cost for this stock
            #     else:
            #         # If no price is available, skip this ticker
            #         total_costs.append(0)
            #         shares_purchased.append(0)
            
            # # Calculate portfolio value (final value of all stocks based on current price)
            # portfolio_value = 0
            # for shares, ticker in zip(shares_purchased, selected_tickers):
            #     if ticker in ticker_prices and ticker_prices[ticker] is not None:
            #         portfolio_value += shares * ticker_prices[ticker]  # Add the value of each stock
            
            # Add the portfolio data to the results
            


In [ ]:
# Function to calculate the purchasing fee depending on the number of shares
def calculate_fee(shares):
    return max(3.95, 0.001*shares)

In [ ]:
# AI

def calculate_fee(shares):
    return max(3.95, 0.001*shares)

def allocate_with_fees(selected_stocks, prices, budget, max_shares=1000000):
    n = len(selected_stocks)
    initial_shares = np.full(n, budget / (n * prices.mean()))  # Initial guess
    
    # Objective: Minimize leftover budget after accounting for fees
    def objective(shares):
        cost = np.sum(prices * shares + np.vectorize(calculate_fee)(prices, shares))
        return abs(budget - cost)
    
    # Constraints: Non-negative shares
    bounds = [(0, max_shares)] * n
    
    # Solve
    result = minimize(objective, initial_shares, bounds=bounds)
    return np.round(result.x)  # Return rounded number of shares

In [ ]:
# AI
def create_portfolio_with_fees(index_ticker="^GSPC", num_stocks=12, budget=1_000_000):
    # Step 1: Benchmark data
    benchmark = yf.Ticker(index_ticker)
    stock_list = pd.DataFrame(benchmark. ())['Symbol'].sample(num_stocks)
    
    # Step 2: Historical prices
    prices = {ticker: yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1] for ticker in stock_list}
    prices = pd.Series(prices)
    
    # Step 3: Optimize shares with fees
    shares = allocate_with_fees(stock_list, prices, budget)
    
    # Step 4: Create portfolio
    portfolio = pd.DataFrame({
        "Stock": stock_list,
        "Price (CAD)": prices.values,
        "Shares": shares,
        "Investment (CAD)": prices.values * shares,
        "Fee (CAD)": [calculate_fee(price, share) for price, share in zip(prices.values, shares)]
    })
    
    portfolio["Total Cost (CAD)"] = portfolio["Investment (CAD)"] + portfolio["Fee (CAD)"]
    portfolio = portfolio[portfolio["Shares"] > 0]  # Remove zero-share stocks
    
    return portfolio, portfolio["Total Cost (CAD)"].sum()

In [138]:
portfolio_data

[]

In [213]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def generate_portfolio(stock_df, market_return, budget, fee_min=3.95, fee_rate=0.001):
    """
    Generate a portfolio that matches the average market return.

    Parameters:
    - stock_df (pd.DataFrame): DataFrame with columns ['Stock', 'MeanReturn', 'StdDev', 'Price'].
    - market_return (float): Target average market return.
    - budget (float): Total budget to allocate to stocks.
    - fee_min (float): Minimum fee per transaction.
    - fee_rate (float): Fee rate per share purchased.

    Returns:
    - dict: A dictionary with stock allocations and total cost.
    """
    # Extract relevant columns
    returns = stock_df['MeanReturn'].values
    prices = stock_df['Price'].values
    num_stocks = len(returns)

    # Objective: minimize total cost (negative return maximization for the problem setup)
    c = prices

    # Constraints
    A_eq = [
        returns,               # Ensure portfolio matches the market return
        np.ones(num_stocks)    # Ensure budget constraint
    ]
    b_eq = [market_return, budget]

    bounds = [(0, None) for _ in range(num_stocks)]  # No short-selling, no upper limit

    # Solve the linear program
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
        allocations = result.x
        total_cost = np.dot(allocations, prices)
        fees = np.sum(np.maximum(fee_min, fee_rate * allocations))
        return {
            'Allocations': {stock: alloc for stock, alloc in zip(stock_df['Stock'], allocations)},
            'TotalCost': total_cost,
            'Fees': fees,
            'PortfolioReturn': np.dot(allocations, returns) / total_cost,
        }
    else:
        raise ValueError("Optimization failed to find a valid portfolio.")
        
# Sample stock data
data = {
    'Stock': ['AAPL', 'MSFT', 'GOOG', 'TSLA'],
    'MeanReturn': [0.12, 0.08, 0.15, 0.10],
    'StdDev': [0.20, 0.25, 0.30, 0.35],
    'Price': [150, 300, 2800, 700]
}
stock_df = pd.DataFrame(data)

market_return = 0.10  # Target market return
budget = 1_000_000   # Budget in dollars

portfolio = generate_portfolio(stock_df, market_return, budget)
print(portfolio)



ValueError: Optimization failed to find a valid portfolio.

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.